In [1]:
from modules.shared.globals import *
from modules.training.cme_modeling import ModelBuilder
from modules.training.ts_modeling import (
    build_dataset,
    load_stratified_folds,
    set_seed)
import numpy as np
from modules.training.ts_modeling import get_plus_cls, get_zero_cls, get_minus_cls, convert_to_onehot_cls
mb = ModelBuilder()

C:\Users\the_3\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\the_3\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If 

In [3]:
seed = SEEDS[0]
inputs_to_use = INPUTS_TO_USE[0]
add_slope = ADD_SLOPE[0]
cme_speed_threshold = CME_SPEED_THRESHOLD[0]
outputs_to_use = OUTPUTS_TO_USE

set_seed(seed)

# set the root directory
root_dir = 'C:/Users/the_3/Documents/github/keras-functional-api/data/electron_cme_data_split_v8'
# root_dir = 'D:/College/Fall2023/sep-forecasting-research/data/electron_cme_data_split_v8'
# build the dataset
X_train, y_train, _, _ = build_dataset(
    root_dir + '/training',
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold,
    shuffle_data=True)

X_test, y_test, _, _ = build_dataset(
    root_dir + '/testing',
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold)

folds = []
for fold_data in load_stratified_folds(
    root_dir,
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold,
    seed=seed,
    shuffle=True
):
    folds.append(fold_data)

# Print training and test shapes
print("Training and Test Shapes:")
print(f'X_train.shape: {X_train.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'X_test.shape: {X_test.shape}') 
print(f'y_test.shape: {y_test.shape}')
print()

# Print shapes for each fold
for i, (X_sub, y_sub, X_v, y_v) in enumerate(folds):
    print(f'Fold {i+1}:')
    print(f'X_subtrain.shape: {X_sub.shape}')
    print(f'y_subtrain.shape: {y_sub.shape}')
    print(f'X_val.shape: {X_v.shape}')
    print(f'y_val.shape: {y_v.shape}')
    print()



Training and Test Shapes:
X_train.shape: (16720, 124)
y_train.shape: (16720, 1)
X_test.shape: (11839, 124)
y_test.shape: (11839, 1)

Fold 1:
X_subtrain.shape: (11955, 124)
y_subtrain.shape: (11955, 1)
X_val.shape: (4765, 124)
y_val.shape: (4765, 1)

Fold 2:
X_subtrain.shape: (12177, 124)
y_subtrain.shape: (12177, 1)
X_val.shape: (4543, 124)
y_val.shape: (4543, 1)

Fold 3:
X_subtrain.shape: (11855, 124)
y_subtrain.shape: (11855, 1)
X_val.shape: (4865, 124)
y_val.shape: (4865, 1)

Fold 4:
X_subtrain.shape: (14173, 124)
y_subtrain.shape: (14173, 1)
X_val.shape: (2547, 124)
y_val.shape: (2547, 1)



In [4]:


# Get plus class data for train and test sets
X_train_plus, y_train_plus = get_plus_cls(X_train, y_train, UPPER_THRESHOLD)
X_test_plus, y_test_plus = get_plus_cls(X_test, y_test, UPPER_THRESHOLD)

print("Plus (>= upper threshold) Shapes:")
print(f'X_train_plus.shape: {X_train_plus.shape}')
print(f'y_train_plus.shape: {y_train_plus.shape}')
print(f'X_test_plus.shape: {X_test_plus.shape}')
print(f'y_test_plus.shape: {y_test_plus.shape}')
print()

# Get plus shapes for each fold
print("Plus shapes for each fold:")
for i, (X_sub, y_sub, X_v, y_v) in enumerate(folds):
    # Get plus data for subtrain and validation sets
    X_sub_plus, y_sub_plus = get_plus_cls(X_sub, y_sub, UPPER_THRESHOLD)
    X_v_plus, y_v_plus = get_plus_cls(X_v, y_v, UPPER_THRESHOLD)
    
    print(f'Fold {i+1}:')
    print(f'X_subtrain_plus.shape: {X_sub_plus.shape}')
    print(f'y_subtrain_plus.shape: {y_sub_plus.shape}')
    print(f'X_val_plus.shape: {X_v_plus.shape}')
    print(f'y_val_plus.shape: {y_v_plus.shape}')
    print()


Plus (>= upper threshold) Shapes:
X_train_plus.shape: (920, 124)
y_train_plus.shape: (920, 1)
X_test_plus.shape: (424, 124)
y_test_plus.shape: (424, 1)

Plus shapes for each fold:
Fold 1:
X_subtrain_plus.shape: (656, 124)
y_subtrain_plus.shape: (656, 1)
X_val_plus.shape: (264, 124)
y_val_plus.shape: (264, 1)

Fold 2:
X_subtrain_plus.shape: (785, 124)
y_subtrain_plus.shape: (785, 1)
X_val_plus.shape: (135, 124)
y_val_plus.shape: (135, 1)

Fold 3:
X_subtrain_plus.shape: (600, 124)
y_subtrain_plus.shape: (600, 1)
X_val_plus.shape: (320, 124)
y_val_plus.shape: (320, 1)

Fold 4:
X_subtrain_plus.shape: (719, 124)
y_subtrain_plus.shape: (719, 1)
X_val_plus.shape: (201, 124)
y_val_plus.shape: (201, 1)



In [5]:


# Get zero class data for train and test sets
X_train_zero, y_train_zero = get_zero_cls(X_train, y_train, LOWER_THRESHOLD, UPPER_THRESHOLD)
X_test_zero, y_test_zero = get_zero_cls(X_test, y_test, LOWER_THRESHOLD, UPPER_THRESHOLD)

print("Zero (between thresholds) Shapes:")
print(f'X_train_zero.shape: {X_train_zero.shape}')
print(f'y_train_zero.shape: {y_train_zero.shape}')
print(f'X_test_zero.shape: {X_test_zero.shape}')
print(f'y_test_zero.shape: {y_test_zero.shape}')
print()

# Get zero shapes for each fold
print("Zero shapes for each fold:")
for i, (X_sub, y_sub, X_v, y_v) in enumerate(folds):
    # Get zero data for subtrain and validation sets
    X_sub_zero, y_sub_zero = get_zero_cls(X_sub, y_sub, LOWER_THRESHOLD, UPPER_THRESHOLD)
    X_v_zero, y_v_zero = get_zero_cls(X_v, y_v, LOWER_THRESHOLD, UPPER_THRESHOLD)
    
    print(f'Fold {i+1}:')
    print(f'X_subtrain_zero.shape: {X_sub_zero.shape}')
    print(f'y_subtrain_zero.shape: {y_sub_zero.shape}')
    print(f'X_val_zero.shape: {X_v_zero.shape}')
    print(f'y_val_zero.shape: {y_v_zero.shape}')
    print()


Zero (between thresholds) Shapes:
X_train_zero.shape: (15070, 124)
y_train_zero.shape: (15070, 1)
X_test_zero.shape: (11105, 124)
y_test_zero.shape: (11105, 1)

Zero shapes for each fold:
Fold 1:
X_subtrain_zero.shape: (10793, 124)
y_subtrain_zero.shape: (10793, 1)
X_val_zero.shape: (4277, 124)
y_val_zero.shape: (4277, 1)

Fold 2:
X_subtrain_zero.shape: (10768, 124)
y_subtrain_zero.shape: (10768, 1)
X_val_zero.shape: (4302, 124)
y_val_zero.shape: (4302, 1)

Fold 3:
X_subtrain_zero.shape: (10764, 124)
y_subtrain_zero.shape: (10764, 1)
X_val_zero.shape: (4306, 124)
y_val_zero.shape: (4306, 1)

Fold 4:
X_subtrain_zero.shape: (12885, 124)
y_subtrain_zero.shape: (12885, 1)
X_val_zero.shape: (2185, 124)
y_val_zero.shape: (2185, 1)



In [6]:


# Get minus class data for train and test sets
X_train_minus, y_train_minus = get_minus_cls(X_train, y_train, LOWER_THRESHOLD)
X_test_minus, y_test_minus = get_minus_cls(X_test, y_test, LOWER_THRESHOLD)

print("Minus (below lower threshold) Shapes:")
print(f'X_train_minus.shape: {X_train_minus.shape}')
print(f'y_train_minus.shape: {y_train_minus.shape}')
print(f'X_test_minus.shape: {X_test_minus.shape}')
print(f'y_test_minus.shape: {y_test_minus.shape}')
print()

# Get minus shapes for each fold
print("Minus shapes for each fold:")
for i, (X_sub, y_sub, X_v, y_v) in enumerate(folds):
    # Get minus data for subtrain and validation sets
    X_sub_minus, y_sub_minus = get_minus_cls(X_sub, y_sub, LOWER_THRESHOLD)
    X_v_minus, y_v_minus = get_minus_cls(X_v, y_v, LOWER_THRESHOLD)
    
    print(f'Fold {i+1}:')
    print(f'X_subtrain_minus.shape: {X_sub_minus.shape}')
    print(f'y_subtrain_minus.shape: {y_sub_minus.shape}')
    print(f'X_val_minus.shape: {X_v_minus.shape}')
    print(f'y_val_minus.shape: {y_v_minus.shape}')
    print()


Minus (below lower threshold) Shapes:
X_train_minus.shape: (730, 124)
y_train_minus.shape: (730, 1)
X_test_minus.shape: (310, 124)
y_test_minus.shape: (310, 1)

Minus shapes for each fold:
Fold 1:
X_subtrain_minus.shape: (506, 124)
y_subtrain_minus.shape: (506, 1)
X_val_minus.shape: (224, 124)
y_val_minus.shape: (224, 1)

Fold 2:
X_subtrain_minus.shape: (624, 124)
y_subtrain_minus.shape: (624, 1)
X_val_minus.shape: (106, 124)
y_val_minus.shape: (106, 1)

Fold 3:
X_subtrain_minus.shape: (491, 124)
y_subtrain_minus.shape: (491, 1)
X_val_minus.shape: (239, 124)
y_val_minus.shape: (239, 1)

Fold 4:
X_subtrain_minus.shape: (569, 124)
y_subtrain_minus.shape: (569, 1)
X_val_minus.shape: (161, 124)
y_val_minus.shape: (161, 1)



In [2]:


# Set seed and parameters
seed = SEEDS[0]
inputs_to_use = INPUTS_TO_USE[0] 
add_slope = ADD_SLOPE[0]
cme_speed_threshold = CME_SPEED_THRESHOLD[0]
outputs_to_use = OUTPUTS_TO_USE

set_seed(seed)

# Set root directory
root_dir = 'C:/Users/the_3/Documents/github/keras-functional-api/data/electron_cme_data_split_v8'

# Build training dataset
X_train, y_train, _, _ = build_dataset(
    root_dir + '/training',
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold,
    shuffle_data=True)

# Build test dataset
X_test, y_test, _, _ = build_dataset(
    root_dir + '/testing',
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold)

# Convert labels to one-hot vectors
y_train_classes = convert_to_onehot_cls(y_train, LOWER_THRESHOLD, UPPER_THRESHOLD)
y_test_classes = convert_to_onehot_cls(y_test, LOWER_THRESHOLD, UPPER_THRESHOLD)

# Print shapes for training and test data
print("Training and Test Shapes with One-Hot Labels:")
print(f'X_train.shape: {X_train.shape}')
print(f'y_train_classes.shape: {y_train_classes.shape}')
print(f'X_test.shape: {X_test.shape}')
print(f'y_test_classes.shape: {y_test_classes.shape}')
print()

# Process folds with one-hot encoded labels
print("Fold Shapes with One-Hot Labels:")
for i, (X_sub, y_sub, X_v, y_v) in enumerate(load_stratified_folds(
    root_dir,
    inputs_to_use=inputs_to_use,
    add_slope=add_slope,
    outputs_to_use=outputs_to_use,
    cme_speed_threshold=cme_speed_threshold,
    seed=seed,
    shuffle=True
)):
    # Convert fold labels to one-hot
    y_sub_classes = convert_to_onehot_cls(y_sub, LOWER_THRESHOLD, UPPER_THRESHOLD)
    y_v_classes = convert_to_onehot_cls(y_v, LOWER_THRESHOLD, UPPER_THRESHOLD)

    print(f'Fold {i+1}:')
    print(f'X_subtrain.shape: {X_sub.shape}')
    print(f'y_subtrain_classes.shape: {y_sub_classes.shape}')
    print(f'X_val.shape: {X_v.shape}')
    print(f'y_val_classes.shape: {y_v_classes.shape}')
    print()


Training and Test Shapes with One-Hot Labels:
X_train.shape: (16720, 124)
y_train_classes.shape: (16720, 3)
X_test.shape: (11839, 124)
y_test_classes.shape: (11839, 3)

Fold Shapes with One-Hot Labels:
Fold 1:
X_subtrain.shape: (11955, 124)
y_subtrain_classes.shape: (11955, 3)
X_val.shape: (4765, 124)
y_val_classes.shape: (4765, 3)

Fold 2:
X_subtrain.shape: (12177, 124)
y_subtrain_classes.shape: (12177, 3)
X_val.shape: (4543, 124)
y_val_classes.shape: (4543, 3)

Fold 3:
X_subtrain.shape: (11855, 124)
y_subtrain_classes.shape: (11855, 3)
X_val.shape: (4865, 124)
y_val_classes.shape: (4865, 3)

Fold 4:
X_subtrain.shape: (14173, 124)
y_subtrain_classes.shape: (14173, 3)
X_val.shape: (2547, 124)
y_val_classes.shape: (2547, 3)



In [3]:
# print the first 3 elements of training one hot
print(y_train_classes[:3])
print(y_train[:3])

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[-0.32598616]
 [ 0.24692117]
 [-0.02931396]]


In [3]:
from modules.training.ts_modeling import stratified_4fold_split


ModuleNotFoundError: No module named 'modules'

In [ ]:
for i, (X_subtrain, y_subtrain, X_val, y_val) in enumerate(stratified_4fold_split(X_train, y_train, seed=seed, debug=True)):
    print(f'Fold {i + 1}:')
    print(f'X_subtrain.shape: {X_subtrain.shape}')
    print(f'y_subtrain.shape: {y_subtrain.shape}')
    print(f'X_val.shape: {X_val.shape}')
    print(f'y_val.shape: {y_val.shape}')

In [ ]:
from scipy.spatial.distance import jensenshannon
import numpy as np
from typing import Dict, Union


def calculate_js_divergence(X1: np.ndarray, X2: np.ndarray) -> Dict[str, float]:
    """
    Calculate the Jensen-Shannon (JS) divergence for each feature between two datasets.

    Args:
        X1 (np.ndarray): The first dataset (e.g., training data) with shape (n_samples, n_features).
        X2 (np.ndarray): The second dataset (e.g., test data) with shape (n_samples, n_features).

    Returns:
        Dict[str, float]: A dictionary where the keys are feature names (or indices) 
                          and the values are the corresponding JS divergence values.
    """
    js_divergences = {}
    n_features = X1.shape[1]  # Number of features

    # Iterate over each feature index
    for i in range(n_features):
        # Create a histogram for the feature in each dataset
        p = np.histogram(X1[:, i], bins=100, density=True)[0]
        q = np.histogram(X2[:, i], bins=100, density=True)[0]

        # Calculate the JS divergence between the two histograms
        js_div = jensenshannon(p, q)

        # Store the JS divergence in the dictionary with the feature index as the key
        js_divergences[f'Feature_{i}'] = js_div

    return js_divergences


import matplotlib.pyplot as plt


def calculate_js_divergence_labels(y1: Union[np.ndarray, list], y2: Union[np.ndarray, list], bin_width: float = 0.1,
                                   plot: bool = False) -> float:
    """
    Calculate the Jensen-Shannon (JS) divergence between the label distributions of two regression datasets.
    Optionally, plot the histograms of the label distributions.

    Args:
        y1 (Union[np.ndarray, list]): The labels for the first dataset (e.g., training labels).
        y2 (Union[np.ndarray, list]): The labels for the second dataset (e.g., test labels).
        plot (bool): If True, plot the histograms of the label distributions. Default is False.

    Returns:
        float: The JS divergence between the label distributions.
    """
    # Define the bin edges for the histograms with width 0.1 between -2.5 and 2.5
    bins = np.arange(-2.5, 2.5 + bin_width, bin_width)

    # Calculate histograms for the labels with the same bins for both datasets
    p = np.histogram(y1, bins=bins, density=True)[0]
    q = np.histogram(y2, bins=bins, density=True)[0]

    # Calculate the JS divergence between the two label distributions
    js_div = jensenshannon(p, q)

    # Plot the histograms if requested
    if plot:
        plt.figure(figsize=(10, 6))

        # Plot the histogram for the first dataset
        plt.hist(y1, bins=bins, density=True, alpha=0.5, color='blue', label='Dataset 1 (y1)')

        # Plot the histogram for the second dataset
        plt.hist(y2, bins=bins, density=True, alpha=0.5, color='orange', label='Dataset 2 (y2)')

        # Adding labels and title
        plt.xlabel('Label Value')
        plt.ylabel('Density')
        plt.title('Label Distribution Comparison')
        plt.legend(loc='upper right')

        # Show the plot
        plt.show()

    return js_div

In [ ]:
# Calculate JS divergence for features
js_train_test = calculate_js_divergence(X_train, X_test)
js_subtrain_test = calculate_js_divergence(X_subtrain, X_test)
js_val_test = calculate_js_divergence(X_val, X_test)

# Calculate JS divergence for labels
js_labels_train_test = calculate_js_divergence_labels(y_train, y_test, bin_width=0.01, plot=True)
js_labels_subtrain_test = calculate_js_divergence_labels(y_subtrain, y_test, bin_width=0.01, plot=True)
js_labels_val_test = calculate_js_divergence_labels(y_val, y_test, bin_width=0.01, plot=True)

# Print all JS divergences for features
print(f'Seed: {seed}, Inputs: {inputs_to_use}, Add Slope: {add_slope}, CME Speed Threshold: {cme_speed_threshold}')
print('JS Divergence between X_train and X_test:')
for feature, js_div in js_train_test.items():
    print(f'Feature: {feature}, JS Divergence: {js_div:.4f}')

print('JS Divergence between X_subtrain and X_test:')
for feature, js_div in js_subtrain_test.items():
    print(f'Feature: {feature}, JS Divergence: {js_div:.4f}')

print('JS Divergence between X_val and X_test:')
for feature, js_div in js_val_test.items():
    print(f'Feature: {feature}, JS Divergence: {js_div:.4f}')

# Print average JS divergences for features
avg_js_train_test = np.mean(list(js_train_test.values()))
avg_js_subtrain_test = np.mean(list(js_subtrain_test.values()))
avg_js_val_test = np.mean(list(js_val_test.values()))
print(f'Average JS Divergence between X_train and X_test: {avg_js_train_test:.4f}')
print(f'Average JS Divergence between X_subtrain and X_test: {avg_js_subtrain_test:.4f}')
print(f'Average JS Divergence between X_val and X_test: {avg_js_val_test:.4f}')

# Print JS divergences for labels
print(f'JS Divergence between y_train and y_test: {js_labels_train_test:.4f}')
print(f'JS Divergence between y_subtrain and y_test: {js_labels_subtrain_test:.4f}')
print(f'JS Divergence between y_val and y_test: {js_labels_val_test:.4f}')

### Summary: Jensen-Shannon (JS) Divergence

- **JS Divergence**: Measures the similarity between two probability distributions.
- **Range**: 0 (identical) to 1 (completely dissimilar).
- **Good Value**: JS close to 0 indicates similar distributions.
- **Bad Value**: JS close to 1 indicates significant differences.

### Results for Your Dataset

- **y_train vs. y_test**: JS = 0.0861
- **y_subtrain vs. y_test**: JS = 0.0867
- **y_val vs. y_test**: JS = 0.0943

These relatively low JS values suggest that the label distributions in your training, subtraining, validation, and test sets are quite similar. While the divergences are not negligible, they are still well within a range that generally indicates good distributional alignment. This implies that your model should be able to generalize well across these datasets, with minimal risk of performance degradation due to distributional differences.

In [ ]:
def calculate_js_divergence_labels_subset(y1: Union[np.ndarray, list], y2: Union[np.ndarray, list],
                                          bin_width: float = 0.1, lower_b: float = -2.5,
                                          higher_b: float = 2.5, plot: bool = False) -> float:
    """
    Calculate the Jensen-Shannon (JS) divergence between the label distributions of two regression datasets
    within a specified subset range. Optionally, plot the histograms of the label distributions.

    Args:
        y1 (Union[np.ndarray, list]): The labels for the first dataset (e.g., training labels).
        y2 (Union[np.ndarray, list]): The labels for the second dataset (e.g., test labels).
        bin_width (float): The width of the bins for the histograms. Default is 0.1.
        lower_b (float): The lower bound of the subset range to consider. Default is -2.5.
        higher_b (float): The upper bound of the subset range to consider. Default is 2.5.
        plot (bool): If True, plot the histograms of the label distributions. Default is False.

    Returns:
        float: The JS divergence between the label distributions within the specified range.
    """
    # Filter the data to consider only the subset within the specified range
    y1_subset = np.array([y for y in y1 if lower_b <= y <= higher_b])
    y2_subset = np.array([y for y in y2 if lower_b <= y <= higher_b])

    # Define the bin edges for the histograms with the specified bin width within the given range
    bins = np.arange(lower_b, higher_b + bin_width, bin_width)

    # Calculate histograms for the labels with the same bins for both datasets
    p = np.histogram(y1_subset, bins=bins, density=True)[0]
    q = np.histogram(y2_subset, bins=bins, density=True)[0]

    # Calculate the JS divergence between the two label distributions
    js_div = jensenshannon(p, q)

    # Plot the histograms if requested
    if plot:
        plt.figure(figsize=(10, 6))

        # Plot the histogram for the first dataset
        plt.hist(y1_subset, bins=bins, density=True, alpha=0.5, color='blue', label='Dataset 1 (y1)')

        # Plot the histogram for the second dataset
        plt.hist(y2_subset, bins=bins, density=True, alpha=0.5, color='orange', label='Dataset 2 (y2)')

        # Adding labels and title
        plt.xlabel('Label Value')
        plt.ylabel('Density')
        plt.title(f'Label Distribution Comparison (Subset between {lower_b} and {higher_b})')
        plt.legend(loc='upper right')

        # Show the plot
        plt.show()

    return js_div

In [ ]:
lower_bound = 0.5
upper_bound = 2.5

# Calculate JS divergence for labels within the subset range
js_labels_train_test_subset = calculate_js_divergence_labels_subset(y_train, y_test, lower_b=lower_bound,
                                                                    higher_b=upper_bound, bin_width=0.01, plot=True)
js_labels_subtrain_test_subset = calculate_js_divergence_labels_subset(y_subtrain, y_test, lower_b=lower_bound,
                                                                       higher_b=upper_bound, bin_width=0.01, plot=True)
js_labels_val_test_subset = calculate_js_divergence_labels_subset(y_val, y_test, lower_b=lower_bound,
                                                                  higher_b=upper_bound, bin_width=0.01, plot=True)

# Print JS divergences for labels within the subset range
print(
    f'JS Divergence between y_train and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_train_test_subset:.4f}')
print(
    f'JS Divergence between y_subtrain and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_subtrain_test_subset:.4f}')
print(
    f'JS Divergence between y_val and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_val_test_subset:.4f}')

In [ ]:
lower_bound = 0.5
upper_bound = 2.5

# Calculate JS divergence for labels within the subset range
# js_labels_train_test_subset = calculate_js_divergence_labels_subset(y_train, y_test, lower_b=lower_bound,
#                                                                     higher_b=upper_bound, bin_width=0.01, plot=True)
# js_labels_subtrain_test_subset = calculate_js_divergence_labels_subset(y_subtrain, y_test, lower_b=lower_bound,
#                                                                        higher_b=upper_bound, bin_width=0.01, plot=True)
js_labels_val_test_subset = calculate_js_divergence_labels_subset(y_val, y_train, lower_b=lower_bound,
                                                                  higher_b=upper_bound, bin_width=0.1, plot=True)


# Print JS divergences for labels within the subset range
# print(
#     f'JS Divergence between y_train and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_train_test_subset:.4f}')
# print(
#     f'JS Divergence between y_subtrain and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_subtrain_test_subset:.4f}')
print(
    f'JS Divergence between y_val and y_train (Subset between {lower_bound} and {upper_bound}): {js_labels_val_test_subset:.4f}')

In [ ]:
lower_bound = -2.5
upper_bound = 2.5

# Calculate JS divergence for labels within the subset range
# js_labels_train_test_subset = calculate_js_divergence_labels_subset(y_train, y_test, lower_b=lower_bound,
#                                                                     higher_b=upper_bound, bin_width=0.01, plot=True)
# js_labels_subtrain_test_subset = calculate_js_divergence_labels_subset(y_subtrain, y_test, lower_b=lower_bound,
#                                                                        higher_b=upper_bound, bin_width=0.01, plot=True)
js_labels_val_test_subset = calculate_js_divergence_labels_subset(y_val, y_train, lower_b=lower_bound,
                                                                  higher_b=upper_bound, bin_width=0.01, plot=True)


# Print JS divergences for labels within the subset range
# print(
#     f'JS Divergence between y_train and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_train_test_subset:.4f}')
# print(
#     f'JS Divergence between y_subtrain and y_test (Subset between {lower_bound} and {upper_bound}): {js_labels_subtrain_test_subset:.4f}')
print(
    f'JS Divergence between y_val and y_train (Subset between {lower_bound} and {upper_bound}): {js_labels_val_test_subset:.4f}')

### Analysis of JS Divergence for the Subset between 0.5 and 2.5

- **JS Divergence between `y_train` and `y_test` (Subset 0.5 to 2.5): 0.4015**
- **JS Divergence between `y_subtrain` and `y_test` (Subset 0.5 to 2.5): 0.3993**
- **JS Divergence between `y_val` and `y_test` (Subset 0.5 to 2.5): 0.5027**

### Interpretation:

- **Higher JS Divergence**:
  - These JS divergence values are notably higher than those observed for the full dataset range. Specifically, the divergence values range from approximately 0.4 to 0.5.
  - **0.4015** and **0.3993** indicate a moderate difference between the training, subtraining, and test label distributions.
  - **0.5027** is quite high, particularly for the validation set compared to the test set, suggesting a more significant difference in this subset range.

### Implications:

- **Potential Distributional Shift**: The higher JS divergence in this specific range (0.5 to 2.5) suggests that there is a notable difference in how the labels are distributed in this region between your training, subtraining, validation, and test sets. This could indicate a distributional shift, particularly in the validation set, that may affect model performance in this range.
- **Impact on Model Performance**:
  - If this range (0.5 to 2.5) is particularly important for your prediction task, the model might struggle to generalize well in this interval due to the distributional differences.
  - The validation set having the highest divergence (**0.5027**) compared to the test set could lead to validation performance not accurately reflecting the test performance.

#

In [ ]:
lower_bound = -2.5
higher_bound = -0.5

# Calculate JS divergence for labels within the subset range
js_labels_train_test_subset = calculate_js_divergence_labels_subset(y_train, y_test, lower_b=lower_bound,
                                                                    higher_b=higher_bound, bin_width=0.01, plot=True)
js_labels_subtrain_test_subset = calculate_js_divergence_labels_subset(y_subtrain, y_test, lower_b=lower_bound,
                                                                       higher_b=higher_bound, bin_width=0.01, plot=True)
js_labels_val_test_subset = calculate_js_divergence_labels_subset(y_val, y_test, lower_b=lower_bound,
                                                                    higher_b=higher_bound, bin_width=0.01, plot=True)

# Print JS divergences for labels within the subset range
print(
    f'JS Divergence between y_train and y_test (Subset between {lower_bound} and {higher_bound}): {js_labels_train_test_subset:.4f}')
print(
    f'JS Divergence between y_subtrain and y_test (Subset between {lower_bound} and {higher_bound}): {js_labels_subtrain_test_subset:.4f}')
print(
    f'JS Divergence between y_val and y_test (Subset between {lower_bound} and {higher_bound}): {js_labels_val_test_subset:.4f}')



### Analysis of JS Divergence for the Subset between -2.5 and -0.5

- **JS Divergence between `y_train` and `y_test` (Subset -2.5 to -0.5): 0.4991**
- **JS Divergence between `y_subtrain` and `y_test` (Subset -2.5 to -0.5): 0.5104**
- **JS Divergence between `y_val` and `y_test` (Subset -2.5 to -0.5): 0.5238**

### Interpretation:

- **High JS Divergence**:
  - These JS divergence values are quite high, close to or exceeding 0.5, which indicates significant differences in the label distributions between your datasets in the subset range from -2.5 to -0.5.
  - **0.4991** for `y_train` vs. `y_test` and **0.5104** for `y_subtrain` vs. `y_test` suggest that the training and subtraining sets differ notably from the test set in this range.
  - **0.5238** for `y_val` vs. `y_test` is the highest, indicating that the validation set is particularly different from the test set in this range.

### Implications:

- **Significant Distributional Shift**: The JS divergence values above 0.5 suggest that there is a substantial shift in label distributions between your datasets in the interval from -2.5 to -0.5. This is a strong indicator that the model trained on these data might not generalize well to the test set, especially in this specific range.
- **Model Performance Concerns**:
  - The model's performance in the range from -2.5 to -0.5 could be severely affected due to these high divergence values. Predictions in this region might be less accurate, as the training and validation sets do not represent the test set well in this interval.
  - The highest divergence in the validation set (**0.5238**) suggests that validation performance may not be a reliable indicator of test performance in this range, potentially leading to over- or under-estimation of the model's capability.

# Debugging of stratified batch sampling

In [ ]:
from modules.training.ts_modeling import stratified_groups, stratified_data_generator, stratified_batch_dataset

In [ ]:
val_groups = stratified_groups(y_val, batch_size=4096)

In [ ]:
# print the size of the whole groups and each group
print(f'Shape of the whole groups: {val_groups.shape}')
print(f'Total number of groups: {len(val_groups)}')
print(f'Sizes of each group: {[len(group) for group in val_groups]}')

In [ ]:
# print the first 5 elements of each group
for i, group in enumerate(val_groups):
    print(f'Group {i}: {group[:5]}')

In [ ]:
# print last 5 elements of each group
for i, group in enumerate(val_groups):
    print(f'Group {i}: {group[-5:]}')

In [ ]:
# get the first batch of the stratified data generator
strat_gen = stratified_data_generator(X_val, y_val, groups=val_groups, debug=True)

In [ ]:
X_batch, y_batch = next(strat_gen)

In [ ]:
# print min and max y in batch
import numpy as np
print(f'Min y in batch: {np.min(y_batch)}')
print(f'Max y in batch: {np.max(y_batch)}')

## checking if the peak delta and peak intensity are correlated

In [ ]:
root_dir = 'C:/Users/the_3/Documents/github/keras-functional-api/data/electron_cme_data_split_v7/full'

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_peak_values(directory_path: str) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Extract peak values from proton intensity data files in a directory.
    
    This function processes CSV files containing proton intensity data and extracts:
    - Peak log-transformed proton intensity values
    - Peak delta log intensity values 
    - Event IDs
    
    Args:
        directory_path (str): Path to directory containing proton intensity CSV files
        
    Returns:
        tuple: Contains three numpy arrays (all sorted by event ID):
            - peak_log_intensity_array: Array of peak log-transformed proton intensities
            - peak_delta_array: Array of peak delta log intensities
            - event_indices_array: Array of event IDs
    """
    # Initialize lists to store peak values and event IDs
    peak_log_intensity_list = []
    peak_delta_list = []
    event_indices = []

    # Get list of files ending with '_ie_trim.csv'
    # These files contain the proton intensity data for each event
    file_names = [f for f in os.listdir(directory_path) if f.endswith('_ie_trim.csv')]

    for file_name in file_names:
        # Construct full file path and read CSV
        file_path = os.path.join(directory_path, file_name)
        data = pd.read_csv(file_path)

        # Extract relevant columns from the data
        proton_intensity = data['Proton Intensity']
        delta_log_intensity = data['delta_log_Intensity']
        
        # Get event ID from first row (assumed constant throughout file)
        event_id = data['Event ID'].iloc[0]

        # Transform proton intensity using log1p (log(1+x))
        # This helps handle small values and maintains non-negativity
        log_proton_intensity = np.log1p(proton_intensity)

        # Calculate peak values for this event
        peak_log_intensity = log_proton_intensity.max()
        peak_delta = delta_log_intensity.max()

        # Store values for this event
        peak_log_intensity_list.append(peak_log_intensity)
        peak_delta_list.append(peak_delta)
        event_indices.append(event_id)

    # Convert lists to numpy arrays for efficient processing
    peak_log_intensity_array = np.array(peak_log_intensity_list)
    peak_delta_array = np.array(peak_delta_list)
    event_indices_array = np.array(event_indices)

    # Sort all arrays based on event IDs to ensure consistent ordering
    sorted_indices = np.argsort(event_indices_array)
    peak_log_intensity_array = peak_log_intensity_array[sorted_indices]
    peak_delta_array = peak_delta_array[sorted_indices]
    event_indices_array = event_indices_array[sorted_indices]

    return peak_log_intensity_array, peak_delta_array, event_indices_array

In [ ]:
def normalize_array(arr: np.ndarray) -> np.ndarray:
    """
    Normalize an array to the range [0,1] using min-max normalization.
    
    Args:
        arr (np.ndarray): Input array to normalize
        
    Returns:
        np.ndarray: Normalized array with values scaled to [0,1]
    """
    # Calculate min and max once to avoid repeated computation
    arr_min = arr.min()
    arr_max = arr.max()
    
    # Normalize using min-max scaling formula: (x - min)/(max - min)
    return (arr - arr_min) / (arr_max - arr_min)

In [ ]:
def plot_peak_values(peak_log_intensity: np.ndarray, 
                     peak_delta: np.ndarray, 
                     event_indices: np.ndarray,
                     event_color: bool = False,
                     norm_axis: bool = True) -> None:
    """
    Create a scatter plot comparing peak log proton intensity vs peak delta values.
    
    Args:
        peak_log_intensity (np.ndarray): Array of peak log proton intensity values
        peak_delta (np.ndarray): Array of peak delta values 
        event_indices (np.ndarray): Array of event indices used for coloring the scatter points
        event_color (bool): Whether to color points by event index (default False)
        norm_axis (bool): Whether to normalize the axis values to [0,1] range (default True)
        
    Returns:
        None: Displays the plot using plt.show()
    """
    # Normalize arrays if requested
    x_values = normalize_array(peak_log_intensity) if norm_axis else peak_log_intensity
    y_values = normalize_array(peak_delta) if norm_axis else peak_delta

    # Create figure with reasonable size for visualization
    plt.figure(figsize=(8, 6))
    
    # Create scatter plot with optional event index coloring
    if event_color:
        scatter = plt.scatter(x_values, y_values, 
                            c=event_indices, cmap='viridis')
        # Add colorbar to show event index scale
        plt.colorbar(scatter, label='Event Index')
    else:
        plt.scatter(x_values, y_values)
    
    # Add diagonal reference line if normalized
    if norm_axis:
        plt.plot([0,1], [0,1], linestyle='--', color='gray')
    
    # Label axes and title
    x_label = 'Normalized ' if norm_axis else ''
    y_label = 'Normalized ' if norm_axis else ''
    plt.xlabel(f'{x_label}Peak Log Proton Intensity')
    plt.ylabel(f'{y_label}Peak Delta')
    plt.title('Peak Log Proton Intensity vs Peak Delta')
    
    # Add grid for better readability
    plt.grid(True)
    
    # Display the plot
    plt.show()

In [ ]:
# Usage example
directory_path = root_dir
peak_log_intensity_array, peak_delta_array, event_indices_array = get_peak_values(directory_path)
print("\nEvent Index | Peak Log Intensity | Peak Delta")
print("-" * 45)
for i, (intensity, delta, event) in enumerate(zip(peak_log_intensity_array, peak_delta_array, event_indices_array)):
    print(f"{event:^11d} | {intensity:^17.4f} | {delta:^10.4f}")

plot_peak_values(peak_log_intensity_array, peak_delta_array, event_indices_array)

# Plot unnormalized version
plot_peak_values(peak_log_intensity_array, peak_delta_array, event_indices_array, norm_axis=False)

# Sort events by peak delta in descending order and display
sorted_indices = np.argsort(peak_delta_array)[::-1]
print("\nEvents sorted by peak delta (descending):")
print("Event Index | Peak Delta | Peak Log Intensity")
print("-" * 45)
for idx in sorted_indices:
    event = event_indices_array[idx]
    intensity = peak_log_intensity_array[idx] 
    delta = peak_delta_array[idx]
    print(f"{event:^11d} | {delta:^10.4f} | {intensity:^17.4f}")

# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Original scatter with events colored by sorted index
scatter1 = ax1.scatter(peak_log_intensity_array, peak_delta_array,
                      c=np.arange(len(sorted_indices)), cmap='viridis')
ax1.set_xlabel('Peak Log Proton Intensity')
ax1.set_ylabel('Peak Delta')
ax1.set_title('Events Colored by Original Order')
ax1.grid(True)
plt.colorbar(scatter1, ax=ax1, label='Event Order')

# Plot 2: Scatter with events colored by peak delta value
scatter2 = ax2.scatter(peak_log_intensity_array, peak_delta_array,
                      c=peak_delta_array, cmap='viridis')
ax2.set_xlabel('Peak Log Proton Intensity')
ax2.set_ylabel('Peak Delta')
ax2.set_title('Events Colored by Peak Delta')
ax2.grid(True)
plt.colorbar(scatter2, ax=ax2, label='Peak Delta')

plt.tight_layout()
plt.show()


### Testing if Jesse's plot reduced down to persistent model

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:


def read_sep_data(file_path: str) -> pd.DataFrame:
    """
    Reads the SEP event data from the specified file path.

    Parameters:
    - file_path (str): Path to the SEP event file.

    Returns:
    - pd.DataFrame: The SEP event data as a pandas DataFrame.
    """
    df = pd.read_csv(file_path)
    return df



In [ ]:

def plot_persistent_model(
    test_set_path: str,
    output_file: str = 'persistent_model_with_delta_plot.png',
    colormap: str = 'viridis'
):
    """
    Plots the log proton intensity (actual proton) on the x-axis against the persistent model (p_t_log) on the y-axis,
    with the color being the actual proton intensity using the specified colormap. Also plots the actual delta values
    against the persistent model delta (which is zero), with the color being the actual delta values. Both plots are 
    displayed side by side.

    Parameters:
    - test_set_path (str): Path to the directory containing the test set SEP event files.
    - output_file (str): Filename to save the plot. Default is 'persistent_model_with_delta_plot.png'.
    - colormap (str): Colormap to use for the scatter plots. Default is 'viridis'.

    Returns:
    - None
    """
    # Initialize lists to hold data
    actual_proton_logs = []
    p_t_logs = []
    actual_protons = []
    actual_deltas = []
    persistent_deltas = []

    # Iterate over files in the test_set_path
    for file_name in os.listdir(test_set_path):
        if file_name.endswith('_ie_trim.csv'):
            file_path = os.path.join(test_set_path, file_name)
            try:
                # Read the SEP event data
                df = read_sep_data(file_path)

                # Skip files where proton intensity is -9999
                if (df['Proton Intensity'] == -9999).any():
                    continue

                # Compute actual proton intensity and its log
                actual_proton = df['Proton Intensity'].values
                actual_proton_log = np.log1p(actual_proton)

                # Compute p_t_log (persistent model)
                p_t = df['p_t'].values
                p_t_log = np.log1p(p_t)

                # Get actual delta values from dataset
                delta_actual = df['delta_log_Intensity'].values

                # Persistent model delta is zero (since the persistent model assumes no change)
                delta_persistent = np.zeros_like(delta_actual)

                # Append data to lists
                actual_protons.extend(actual_proton)
                actual_proton_logs.extend(actual_proton_log)
                p_t_logs.extend(p_t_log)
                actual_deltas.extend(delta_actual)
                persistent_deltas.extend(delta_persistent)

            except Exception as e:
                print(f"Error processing file: {file_name}")
                print(e)
                continue

    # Check if data is collected
    if len(actual_proton_logs) == 0:
        print("No data to plot.")
        return

    # Convert lists to numpy arrays
    actual_proton_logs = np.array(actual_proton_logs)
    p_t_logs = np.array(p_t_logs)
    actual_protons = np.array(actual_protons)
    actual_deltas = np.array(actual_deltas)
    persistent_deltas = np.array(persistent_deltas)

    # Calculate the Pearson correlation coefficient for intensities
    correlation_coef_intensity = np.corrcoef(actual_proton_logs, p_t_logs)[0, 1]
    print(f"Pearson correlation coefficient (Intensity): {correlation_coef_intensity:.4f}")

    # Create a figure with two subplots side by side
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Intensity Plot (Left)
    scatter1 = axes[0].scatter(
        actual_proton_logs,
        p_t_logs,
        c=actual_protons,
        cmap=colormap,
        alpha=0.7,
        edgecolors='w',
        linewidth=0.5
    )
    cbar1 = fig.colorbar(scatter1, ax=axes[0])
    cbar1.set_label('Actual Proton Intensity')

    # Add diagonal line representing perfect correlation
    min_val = min(min(actual_proton_logs), min(p_t_logs))
    max_val = max(max(actual_proton_logs), max(p_t_logs))
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Correlation')
    axes[0].legend()

    axes[0].set_xlabel('Actual Proton ln(Intensity)')
    axes[0].set_ylabel('Persistent Model Proton ln(Intensity)')
    axes[0].set_title('Actual vs Persistent Model Proton ln(Intensity)')

    # Add the correlation coefficient text to the plot
    axes[0].text(
        0.05,
        0.95,
        f'Pearson r = {correlation_coef_intensity:.4f}',
        transform=axes[0].transAxes,
        fontsize=12,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.5)
    )

    axes[0].grid(True)

    # Delta Plot (Right)
    scatter2 = axes[1].scatter(
        actual_deltas,
        persistent_deltas,
        c=actual_deltas,
        cmap=colormap,
        alpha=0.7,
        edgecolors='w',
        linewidth=0.5
    )
    cbar2 = fig.colorbar(scatter2, ax=axes[1])
    cbar2.set_label('Actual Delta ln(Intensity)')

    # Add horizontal line at zero (persistent model delta)
    axes[1].axhline(0, color='r', linestyle='--', label='Persistent Model Delta = 0')
    axes[1].legend()

    axes[1].set_xlabel('Actual Delta ln(Intensity)')
    axes[1].set_ylabel('Persistent Model Delta ln(Intensity)')
    axes[1].set_title('Actual Delta vs Persistent Model Delta')

    # Removed statistics text from delta plot

    axes[1].grid(True)

    plt.tight_layout()

    # Save the plot to a file
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"Saved plot to: {output_file}")

    # Show the plot
    plt.show()


In [ ]:
# Replace 'path_to_test_set_directory' with the actual path to your test set directory
root_dir = 'C:/Users/the_3/Documents/github/keras-functional-api/data/electron_cme_data_split_v7'
test_dir = root_dir + '/testing'
plot_persistent_model(test_dir)


### Let's sort the events by peak delta